# Incidence Matrices for quasi-cords

## Classes and functions

### modules

In [93]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets

### Side class, Segment class

In [2]:
class Side:
    '''A side of the square'''
    
    def __init__(self, label, number):
        '''
        Arguments
        ---------
        label : string
            One of the letters in {T,L,B,R}.
        number : int
            The number coressponding to the points on the side of the label.
        '''
        self.label = label
        self.number = number
        self.points = self.get_points()
        
    def get_points(self):
        points = [self.label+str(k)+'+' for k in range(self.number)]
        points += [label[:-1]+'-' for label in reversed(points)]
        if self.label in ['L','R']:
            idx = int(len(points)/2)
            points.insert(idx, self.label)
        return points

class Segment:
    '''A segment on the square connecting two points on the sides'''
    
    def __init__(self, point01, point02):
        '''
        Arguments
        ---------
        point01, point02 : string
            The labels of the two points on the sides.
        '''
        self.ends = (point01, point02)
        
    def show(self):
        return self.ends

### Square class

In [36]:
LABELS = ['T','L','B','R']

class Square:
    '''A square obtained by cut the disk with 4 points T,L,B,R along the arcs a_T, a_L, a_B, a_R'''
    
    def __init__(self, code):
        '''
        Arguments
        ---------
        code : tuple
            The 4-tuple corresponding to the quasi-cord.
        '''
        self.verify_code(code)
        #---
        self.code = code                      # tuple: code = (\d, \d, \d, \d)
        self.sides = self.get_sides()         # dict: sides = {label: Side(label, num),...}
        self.segments = self.get_segments()   # list: segments = [Segment,...]
   
    def verify_code(self, code):
        tl = sum(code[:2])
        br = sum(code[-2:])
        if sum(code[:2]) - sum(code[-2:]) < 0:
            raise ValueError("T+L < B+R")
        elif code[0] - code[1] - br > 0:
            raise ValueError("T > L+B+R")
        elif code[1] - code[0] - br > 0:
            raise ValueError("L > T+B+R")
        else: return True
        
    def get_sides(self):
        sides_dict = {}
        for i in range(len(LABELS)):
            sides_dict[LABELS[i]] = Side(LABELS[i], self.code[i])
        return sides_dict

    def get_segments(self):
        segs_list = []
        
        # create the top-left segments 
        tl = self.sides['T'].number + self.sides['L'].number
        br = self.sides['B'].number + self.sides['R'].number
        diff = tl - br
              #TODO: tl >= br でなかったらエラーを吐くようにする。
        for i in range(diff):
            a_tlseg = Segment(self.sides['T'].points[i], self.sides['L'].points[i])
            segs_list.append(a_tlseg)
        
        # the remaining
        remaining = []
        TL_points = list(reversed(self.sides['L'].points[diff:]))+self.sides['T'].points[diff:]
        BR_points = self.sides['B'].points+self.sides['R'].points

        for i in range(len(TL_points)):
            a_seg = Segment(TL_points[i], BR_points[i])
            segs_list.append(a_seg)
        return segs_list
    
    def matrix(sqr):
        variables = []
        for label in LABELS:
            num = self.sides[label].number
            if label in ['L','R']:
                num += 1
            variables.append(self.sides[label].points[:num])
        variables = [v.replace('+','') for v in sum(variables, [])] # flatten and remove '+'
        #---
        s = len(self.segments)
        M = np.zeros((s,s+1), dtype=int)
        raw = 0
        for a_seg in self.segments:
            i = 0
            for end_point in a_seg.ends:
                if end_point[-1] in ['+','-']:
                    end_point = end_point[:-1]  # remove '+' and '-'
                M[raw][variables.index(end_point)] = (-1)**i
                i += 1
            raw += 1

        tl = self.sides['T'].number + self.sides['L'].number
        C = np.roll(np.eye(1, s+1, dtype=int), shift=tl, axis=1)
        M = np.r_[M, C]
        return M

    def graphic(self, figsize=(400, 400)):
        fig = go.Figure()
        SIZE = 10
    
        #--- Points
        moves = {'T': (1, SIZE*1j), 'L': (-1j, SIZE*(-1)), 'B': (1, SIZE*1j*(-1)), 'R': (1j, SIZE)}
        all_points_dict = {}
        for side in self.sides.values():
            num = side.number
            delta = SIZE/(num+1)
            #---
            complex_list = list(reversed([(-1)*delta*(k+1) for k in range(num)]))
            if side.label in ['L', 'R']:
                complex_list.append(0)
            complex_list += [delta*(k+1) for k in range(num)]
            complex_list = [v*moves[side.label][0]+moves[side.label][1] for v in complex_list]
            for k in range(len(side.points)):
                all_points_dict[side.points[k]] = complex_list[k]
            #---
            fig.add_trace(go.Scatter(
                x = [z.real for z in complex_list],
                y = [z.imag for z in complex_list],
                mode="markers",
            ))
     
        #--- Frame
        frame = go.layout.Shape(
                    type="rect",
                    x0=SIZE, y0=SIZE, x1=(-1)*SIZE, y1=(-1)*SIZE,
                    line=dict(color="RoyalBlue",),
                )
        #--- Segments
        my_shapes = [frame]
        for seg in self.segments:
            pnt0 = all_points_dict[seg.ends[0]]
            pnt1 = all_points_dict[seg.ends[1]]
            my_shapes.append(
                go.layout.Shape(
                    type="line",
                    x0=pnt0.real, y0=pnt0.imag,
                    x1=pnt1.real, y1=pnt1.imag,
                    line=dict(
                        color="LightSeaGreen",
                        width=3,
                    ),
                )
            )
        else: fig.update_layout(shapes=my_shapes)
     
        #--- Update axes properties
        rng = SIZE*1.2
        options = dict(range=[(-1)*rng, rng], showticklabels=False, showgrid=False, zeroline=False,)
        fig.update_xaxes(options)
        fig.update_yaxes(options)
        #--- Set figure size and background color
        fig.update_layout(width=figsize[0], height=figsize[1], plot_bgcolor="white", showlegend=False)
        
        return fig

### show_diagram function

In [49]:
def get_diagram(matrix, figsize=(400,400)):
    fig = go.Figure()
    SIZE = 10
    delta = SIZE/len(matrix)

    def trans(pair):
        return (pair[1] - pair[0]*1j)*delta +(3/2+1/2*1j)*delta + SIZE*1j
    
    #--- Points ---
    corners = dict(points=[], marker=dict(size=5.0, color="Red"),)
    others = dict(points=[], marker=dict(size=2.0, color="RoyalBlue"),)

    entry = np.nditer(M, flags=['multi_index'])
    while not entry.finished:
        if entry[0]==0:
            others['points'].append(entry.multi_index)
        else:
            corners['points'].append(entry.multi_index)
        entry.iternext()

    for pdict in [others, corners]:
        z_list = [trans(pair) for pair in pdict['points']]
        fig.add_trace(go.Scatter(
            x = [z.real for z in z_list],
            y = [z.imag for z in z_list],
            mode="markers",
            marker=pdict['marker'],
        ))

    #--- Lines ---
    lines = []
    for k in range(len(matrix)-1):
        for i in range(2):  # 0: horizontal lines, 1: vertical lines
            z = [trans(pair) for pair in corners['points'] if pair[i] == k]
            if len(z) == 2:
                lines.append(
                    go.layout.Shape(
                        type="line",
                        x0=z[0].real, y0=z[0].imag,
                        x1=z[1].real, y1=z[1].imag,
                        line=dict(color="LightSeaGreen", width=1,),
                    ))
    else: fig.update_layout(shapes=lines)
               
    #--- Update axes properties
    options = dict(
        range=[0, SIZE+2*delta], 
        showticklabels=False,
        showgrid=False,
        zeroline=False,
    )
    fig.update_xaxes(options)
    fig.update_yaxes(options)
    #--- Set figure size and background color
    fig.update_layout(
        width=figsize[0], height=figsize[1],
    #     plot_bgcolor="white", 
        showlegend=False,
    )
    #---
    return fig

### square_random_generator function

In [6]:
def square_random_generator(verbose=True):
    while True:
        try:
            code = [np.random.randint(1,10) for k in range(4)]
            sqr = Square(tuple(code))
            break
        except ValueError:
            if verbose:
                print("Oops!")
    return sqr

## Create an instance of Square class.

In [115]:
sqr = square_random_generator()
print(sqr.code)

Oops!
(6, 9, 5, 7)


### Parts of a square

In [33]:
sqr = square_random_generator()
#sqr = Square((5,1,1,1)) #1,4,1,1)) #0,1,0,0)) #0,3,0,3)) #8,4,6,2)) #3,2,1,1)) #(3,4,5,2)) #(3,7,4,2)) #(1,3,1,1)) #(3,4,2,1)) #

print(sqr.code)

Oops!
Oops!
(2, 9, 7, 4)


End points on the sides

In [8]:
for side in sqr.sides.values():
    print("side {} --> {}".format(side.label, side.points))

side T --> ['T0+', 'T1+', 'T2+', 'T3+', 'T4+', 'T5+', 'T6+', 'T7+', 'T8+', 'T8-', 'T7-', 'T6-', 'T5-', 'T4-', 'T3-', 'T2-', 'T1-', 'T0-']
side L --> ['L0+', 'L1+', 'L2+', 'L3+', 'L', 'L3-', 'L2-', 'L1-', 'L0-']
side B --> ['B0+', 'B1+', 'B1-', 'B0-']
side R --> ['R0+', 'R1+', 'R2+', 'R3+', 'R4+', 'R5+', 'R6+', 'R7+', 'R8+', 'R', 'R8-', 'R7-', 'R6-', 'R5-', 'R4-', 'R3-', 'R2-', 'R1-', 'R0-']


Segments

In [9]:
all_segs = [seg.ends for seg in sqr.segments]
print(all_segs)

[('T0+', 'L0+'), ('T1+', 'L1+'), ('L0-', 'B0+'), ('L1-', 'B1+'), ('L2-', 'B1-'), ('L3-', 'B0-'), ('L', 'R0+'), ('L3+', 'R1+'), ('L2+', 'R2+'), ('T2+', 'R3+'), ('T3+', 'R4+'), ('T4+', 'R5+'), ('T5+', 'R6+'), ('T6+', 'R7+'), ('T7+', 'R8+'), ('T8+', 'R'), ('T8-', 'R8-'), ('T7-', 'R7-'), ('T6-', 'R6-'), ('T5-', 'R5-'), ('T4-', 'R4-'), ('T3-', 'R3-'), ('T2-', 'R2-'), ('T1-', 'R1-'), ('T0-', 'R0-')]


### Graphic

In [52]:
fig = sqr.graphic(figsize=(500,500))
fig.show()

### Incidence Matrix

Create the coefficient matrix of the equations coresponding to the segments

In [42]:
M = get_matrix(sqr)
print("{} --det--> {}".format(M, np.linalg.det(M)))

[[ 0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0 -1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0 -1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1

### Connecting $\pm 1$'s on the matrix

In [51]:
fig = get_diagram(M, figsize=(500,500))
fig.show()

### All in One

In [122]:
sqr = square_random_generator(verbose=False)
M = get_matrix(sqr)
output = "{}\n\n {} |--det--> {}\n\n".format(sqr.code, M, np.linalg.det(M)); print(output)

figs = [get_diagram(M), sqr.graphic()]
fws = [go.FigureWidget(data=fig['data'], layout=fig['layout']) for fig in figs]
display(widgets.HBox(fws))

(2, 9, 4, 7)

 [[ 0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0 -1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0]
 [ 0  0  0

    'data': [{'marker': {'color': 'RoyalBlue', 'size': 2.0},
              'mode…

## Experiments

### Cofactor expansion 

In [101]:
sqr = square_random_generator(verbose=False)
M = get_matrix(sqr)
output = "{}\n\n {} |--det--> {}\n\n".format(sqr.code, M, np.linalg.det(M)); print(output)

figs = [get_diagram(M, figsize=fs), sqr.graphic(figsize=fs)]
fws = [go.FigureWidget(data=fig['data'], layout=fig['layout']) for fig in figs]
display(widgets.HBox(fws))

(4, 7, 1, 6)

 [[ 1  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 -1]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 -1  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  0

    'data': [{'marker': {'color': 'RoyalBlue', 'size': 2.0},
              'mode…

In [ ]:
def cofexp_raw(sqr_matrix, raw_num=0):
    expansion = []
    raw = sqr_matrix[raw_num]
    for j in range(raw.size):
        if raw[j] != 0:
            shrinked = np.delete(np.delete(sqr_matrix, raw_num, 0),j,1)
            sign = (-1)**(raw_num+j)
            expansion.append([sign*raw[j], shrinked])
    return expansion

__予想1:__ 接続行列のどの行(最終行を除く)で余因子展開しても、
全体が正則なら、必ず行列式 $\pm 1$ のものが一つだけ。

これが正しいなら、正則な接続行列の行列式は $\pm 1$ であるとも言える。

__予想2:__ 同様の余因子展開について、非正則ならすべて行列式 $0$.

こちらの予想は反例があった。ex) Square(9,7,9,7), Square(8,4,6,2)

In [ ]:
#sqr = square_random_generator(); 
#sqr = Square((0,3,0,3))
#M = get_matrix(sqr)

print("{} --det--> {}\n".format(sqr.code, np.linalg.det(M)))
# print("{}\n".format(M))

for rn in range(len(sqr.segments)):
    expn = cofexp_raw(M, raw_num=rn)
    values = []
    for pair in expn:
        m = pair[1]
        values.append(int(np.linalg.det(m)))
    else: print("raw {} --raw_exp--> {}".format(rn, values))

__予想2(修正):__ 同様の余因子展開について、非正則なら、どの行でも 2つの行列式を足すと $0\mod 2$.


In [ ]:
def recursive_cofexp_raw(coeff_matrix_pair, count=2):
    if count > 0:
        count += -1
        sqr_matrix = coeff_matrix_pair[1]
#         print(sqr_matrix)
        coeff_matrix_pair[1] = [recursive_cofexp_raw(pair, count) for pair in cofexp_raw(sqr_matrix)]
    return coeff_matrix_pair

In [ ]:
expansion = recursive_cofexp_raw([1,M], count=5)
# print(expansion)

def show_expansion(expn, depth=0):
    for v in expn:
        if isinstance(v, list):
            show_expansion(v, depth+1)
        else:
            if isinstance(v, np.ndarray):
                det = np.linalg.det(v)
                if det != 0:
                    print("{} --> {}".format(v, np.linalg.det(v)))

show_expansion(expansion, depth=3)            

### Elementary deformations

In [ ]:
def elem_matrix(size=1, position=(1,1)):
    i,j = position
    P = np.eye(size, size, dtype=int)
    P[i][j] = 1
    return P
def P(n,ij):
    return elem_matrix(size=n, position=ij)

deform an incidence matrix by elementary matrices

In [ ]:
s = len(sqr.segments)
N = M

num = 0
for side in sqr.sides.values():
    for k in range(side.number):
            N = N@P(s+1,(num+k+1, num+0))
            N = P(s+1,(num+0, num+k+1))@N
    num += side.number

print(N)

### Others

In [ ]:
s = len(sqr.segments)
# N = M
N[s][s]=1
print("{} --det--> {}".format(N, np.linalg.det(N)))

## TODO

* [ ] 接続行列に対して、基本変形を自由にできるようにする。
* [x] show_graphic 関数で、なぜか最初の segment を描画してくれない。
* [x] 行列生成処理も何らかのクラスのメソッドにして、大量計算を実行できるようにしたい。
* [x] 接続行列の生成について。quasi-cord が $\tau + \lambda = \beta + \rho$ の場合しかうまく行ってない。